<a href="https://colab.research.google.com/github/ViniTeider/Threads-Garcons-Cozinheiros/blob/main/FEITO_Atividade_Pr%C3%A1tica_em_Grupo_RA3_2_2023_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Alunos:

* Guilherme Quadro Daudt
* Henrique Alves
* Julia Cristina Moreira
* Vinicius Teider


In [ ]:
import time
import threading

pedidosPendentes = []
pedidosPraEntregar = []
pedidosDoDia = 0
pedidosEntregues = 0

# lock = threading.Lock()

cozinhaDisponivel = threading.Condition()

class Garcom:

  def __init__(self, numeroMaxPedidos, id):
    self.numeroMaxPedidos = numeroMaxPedidos
    self.id = id
  
  def getId(self):
    return self.id

  def fazer_pedido(self):
    global pedidosDoDia, pedidosPendentes

    while self.numeroMaxPedidos != 0:
      with cozinhaDisponivel:
        if len(pedidosPendentes) < 4:

          self.numeroMaxPedidos -= 1
          #with lock:
          pedidosDoDia += 1 
          #print("pedidos do dia " + str(pedidosDoDia))
          pedidosPendentes.append(pedidosDoDia)
          print("Pedido nº{} cadastrado pelo garçom {}".format(pedidosDoDia, self.getId()))   

          cozinhaDisponivel.notify_all()
          #print(pedidosPendentes)

      time.sleep(0.5) 
          
    return
    

class Cozinheiro:
 
  def __init__(self, id):
    self.id = id

  def prepara_pedido(self):
    global pedidosDoDia, pedidosPendentes

    with cozinhaDisponivel:
      while len(pedidosPendentes) > 0:
       
        #print(pedidosPendentes)
        pedidoFeito = pedidosPendentes.pop(0)
        pedidosPraEntregar.append(pedidoFeito)
        print("Pedido nº{} retirado pelo cozinheiro {}".format(pedidoFeito, self.getId()))   
        #print(pedidosPendentes)

        cozinhaDisponivel.notify_all()

      if pedidosDoDia < 40:
        print("cozinheiro esperando")
        with cozinhaDisponivel:
          cozinhaDisponivel.wait()
          self.prepara_pedido()

    time.sleep(1)


  def getId(self):
    return self.id

def execucaoCozinheiroUnico(numeroCozinheiros):


  separador = "*" * 45
  print(separador + f"EXECUTANDO COM {numeroCozinheiros} COZINHEIRO(s)" + separador )
  #inicia o contador de tempo
  tempoInicio = time.time()
  

  garcons = []
  cozinheiros = []
  threads = []

  #criando os 4 cozinheiros
  for id in range(1,5):
    garcom = Garcom(10,id)
    garcons.append(garcom)

  #inicia as threads dos garcons
  for garcom in garcons:
    thread = threading.Thread(target=garcom.fazer_pedido)
    thread.start()
    threads.append(thread)
      
  #cria o numero de cozinheiros dinamicamente
  for id in range(1,numeroCozinheiros+1):
      cozinheiro = Cozinheiro(id)
      cozinheiros.append(cozinheiro)

  #inicia as threads dos cozinheiros
  for cozinheiro in cozinheiros:
    thread = threading.Thread(target=cozinheiro.prepara_pedido)
    thread.start()
    threads.append(thread)

  for thread in threads:
    thread.join()

  tempoFinal = time.time()

  tempoExecucao = tempoFinal - tempoInicio

  print(f"Levou {tempoExecucao} segundos pra executar")
  print("pedidos feitos no dia:" + str(pedidosDoDia))


execucaoCozinheiroUnico(1)


pedidosPendentes = []
pedidosPraEntregar = []
pedidosDoDia = 0
pedidosEntregues = 0

execucaoCozinheiroUnico(2)
